# Višedimenzionalna linearna regresija

Broj pređenih kilometera sigurno utiče na cijenu auta, ali nije ni približno jedini faktor koji ju određuje. Tu su još i starost auta, broj odrađenih servisa, trošnost auta, boja auta itd. Nameće se pitanje, da li možemo našu linearnu regresiju proširiti tako da umjesto jednog parametra, broj pređenih kilometara, ona prima više parametara na osnovu kojih će vršiti procjenu cijene auta. Odgovor je da može, sve dok je skup vrijednosti svakog parametra uređen *(tj. između nas, sve dok možemo vrijednosti bilo kog parametra poredati na brojnu osu)*.

Generalizirajmo malo prethodni navod. Linearna regresija $y = m_1\cdot x + m_0$, se može proširiti na način da umjesto jednog parametra $x$, prima vektor $\overline{x}=(x_1, ..., x_p)$, $x_i \in S_i$, $\forall i \in \overline{1, p}$, gdje je $S_i$ uređen za svako $i \in \overline{1, p}$, $p \in \mathbb{N}$. Takva linearna regresija se naziva **višedimenzionalna linearna regresija**.

Način na koji se vrši navedeno proširenje je poprilično prirodan: $$y = \overline{m}\cdot\overline{x} + m_0$$ s tim da je $\overline{m}$ sada vektor i to takav da $\overline{m} \in \mathbb{R}^p$, a $\overline{m}\cdot\overline{x}$ odgovara skalarnom produktu vektora $\overline{m}$ i $\overline{x}$.

Sjetimo se da nam je kod linearne regresije $y = m_1\cdot x + m_0$ cilj bio da za date parove $(x_i, y_i)$ u trening skupu, odredimo vrijednost parametara $m_1$ i $m_0$, bilo koristeći metaheuristike ili analizu ekstrema površi, takve da prava $y = m_1\cdot x + m_0$ što bolje "oslika" dati trening skup. Ista analogija se primjenjuje i kod višedimenzionalne linearne regresije. Naime, želimo da za date parove $(\overline{x}, y)$ novog trening skupa, pronađemo $\overline{m_1}$ i $m_0$, takve da hiperravan $y = \overline{m}\cdot\overline{x} + m_0$ što bolje "oslika" dati trening skup. Prisjetimo se još da smo kod linearne regresije kvalitet kojim prava "oslikava" trening skup mjerili računajući MSE trening skupa $(x_i, y_i)$: $$MSE(\chi, m_0, m_1) = \frac{1}{|\chi|}\cdot\sum_{(x, y) \in \chi}{(m_1\cdot x + m_0 - y)^2}$$
Odlučimo se da za mjerenje kvaliteta višedimenzionalne linearne regresije, koristimo također MSE: $$MSE(\chi, m_0, \overline{m}) = \frac{1}{|\chi|}\cdot\sum_{(\overline{x}, y) \in \chi}{(\overline{m}\cdot \overline{x} + m_0 - y)^2}$$

Vratimo se našemu konkretnom problemu procjene cijene auta. Recimo da sada, pored broja pređenih kilometara, imamo i informaciju o starosti auta. Dakle naš trening skup se sastoji od uređenih parova $((broj\_pređenih\_kilometara, starost\_auta), cijena\_auta)$. Radi lakše prezentacije podataka, recimo da nam je starost auta data u satima, te da naš prošireni trening skup glasi: $${((10000km, 5000h), 31000\$), ((400000km, 100000h), 19000\$), ((5000km, 2500h), 32000\$), ((0km, 0h), 40000\$), ((1000km, 500h), 33000\$), ((100000km, 50000h), 26000\$), ((50000km, 25000h), 29000\$), ((50km, 24h), 35000\$), ((20000km, 10000h), 30000\$), ((200000km, 100000h), 20000\$)}$$
Zarad lakše prezentacije podataka, dopisane su i jedinice. Iscrtajmo date podatke:

In [1]:
import numpy as np

training_data: np.array = np.array(
    [(10000 ,5000, 31000), (400000, 100000, 19000), (5000, 2500, 32000),
     (0, 0, 40000), (1000, 500, 33000), (100000, 50000, 26000),
     (50000, 25000, 29000), (50, 24, 35000), (20000, 10000, 30000),
     (200000, 100000, 20000)])

In [3]:
import plotly.offline as ply
import plotly.graph_objs as go

ply.init_notebook_mode(connected=True)


car_mileages: np.ndarray = training_data.T[0]
car_ages: np.ndarray = training_data.T[1]
car_values: np.ndarray = training_data.T[2]

plot_data = go.Scatter3d(x=car_mileages, 
                         y=car_ages,
                         z=car_values,
                         mode='markers',
                         marker=dict(
                            size=10,
                            color=car_values,
                            colorscale='Viridis',
                            opacity=0.8))

layout = go.Layout(
    scene = dict(xaxis=dict(title='Mileage',
                            titlefont=dict(color='rgb(200, 200, 200)')),
                 yaxis=dict(title='Age',
                            titlefont=dict(color='rgb(200, 200, 200)')),
                 zaxis=dict(title='Price',
                            titlefont=dict(color='rgb(200, 200, 200)'))))


figure = go.Figure(data=[plot_data], layout=layout)
ply.iplot(figure)

Dimenzija vektora $\overline{x}$, u trening skupu kojeg čine parovi $(\overline{x}, y)$ određuje dimenziju trening skupa. Odatle je navedeni trening skup dvodimenzionalan, pa je time i linearna regresija koju nalazimo za dati trening skup dvodimenzionalna. Drugim riječima, linerna regresija koju nalazimo sada odgovara ravni $y = m_2\cdot x_2 + m_1\cdot x_1 + m_0$, takvoj da $\frac{1}{|\chi|}\cdot\sum_{((x_1, x_2), y) \in \chi}{(m_2\cdot x_2 + m_1\cdot x_1 + m_0 - y)^2}$ prima najmanju moguću vrijednost.

Za početak, iskoristimo biblioteku `sklearn` da pronađemo linearnu regresiju datog trening skupa.

In [5]:
from sklearn import linear_model

# Form the training matrix
car_mileages_and_ages: np.ndarray = np.array(list(zip(car_mileages, car_ages)))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training set
regr.fit(car_mileages_and_ages, car_values)

# The mean squared error
sklearn_mse: float = np.mean((regr.predict(car_mileages_and_ages) - car_values) ** 2)
print(f'Mean squared error: {sklearn_mse}')

# Form plane points
plane: np.array = np.array([regr.predict([[car_mileage, car_age]
                                            for car_mileage in car_mileages])
                              for car_age in car_ages])

sklearn_regression_plane = go.Surface(
    x=car_mileages,
    y=car_ages,
    z=plane)

figure = go.Figure(data=[plot_data, sklearn_regression_plane], layout=layout)
ply.iplot(figure)

Mean squared error: 5443580.650145473


Sada za bilo koji par (broj_pređenih_kilometara, startost) možemo iskoristiti pronađenu dvodimenzionalnu linearnu regresiju za procjenu cijene auta:

In [7]:
test_data: np.ndarray = np.array([(150000, 100000), (250000, 200000), (300000, 200000)])
approximated_values: np.ndarray = regr.predict(test_data)

for (mileage, age), value in zip(test_data, approximated_values):
    print(f'Mileage: {mileage} km and Age: {round(age / 24 / 365)} years -> Value: {round(value)} $')

Mileage: 150000 km and Age: 11.0 years -> Value: 19129.0 $
Mileage: 250000 km and Age: 23.0 years -> Value: 4459.0 $
Mileage: 300000 km and Age: 23.0 years -> Value: 4433.0 $


*Nastavak vježbi dolazi uskoro. U međuvremenu: Šta možete reći o nalasku optimalnih parametara $m_0, m_1 i m_2$ u dvodimenzionalnoj linearnog regresiji $y = m_2\cdot x_2 + m_1\cdot x_1 + m_0$, spram funkcije greške $MSE: \frac{1}{|\chi|}\cdot\sum_{((x_1, x_2), y) \in \chi}{(m_2\cdot x_2 + m_1\cdot x_1 + m_0 - y)^2}$? Možemo li ih pronaći analizom ekstrema površi kao što smo to uradili kod jednodimenzionalne linearne regresije? Ili smo primorani da koristimo neku metaheuristiku?*